Dataset: https://www.kaggle.com/ajaysh/women-apparel-recommendation-engine-amazoncom

In [1]:
#import all the necessary packages.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_json('tops_fashion.json')

In [3]:
df.head()

,asin,author,availability,availability_type,brand,color,editorial_reivew,editorial_review,formatted_price,large_image_url,manufacturer,medium_image_url,model,product_type_name,publisher,reviews,sku,small_image_url,title
0,B016I2TS4W,None,None,None,FNC7C,None,NaN,Minions Como Superheroes Ironman Women's O Nec...,None,https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,None,SHIRT,None,"[False, https://www.amazon.com/reviews/iframe?...",None,https://images-na.ssl-images-amazon.com/images...,Minions Como Superheroes Ironman Long Sleeve R...
1,B01N49AI08,None,None,None,FIG Clothing,None,NaN,Sizing runs on the small side. FIG® recommends...,None,https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,None,SHIRT,None,"[False, https://www.amazon.com/reviews/iframe?...",None,https://images-na.ssl-images-amazon.com/images...,FIG Clothing Womens Izo Tunic
2,B01JDPCOHO,None,None,None,FIG Clothing,None,NaN,Sizing runs on the small side. FIG® recommends...,None,https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,None,SHIRT,None,"[False, https://www.amazon.com/reviews/iframe?...",None,https://images-na.ssl-images-amazon.com/images...,FIG Clothing Womens Won Top
3,B01N19U5H5,None,None,None,Focal18,None,NaN,100% Brand New & Fashion<br> Quantity: 1 Piece...,None,https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,None,SHIRT,None,"[True, https://www.amazon.com/reviews/iframe?a...",None,https://images-na.ssl-images-amazon.com/images...,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...
4,B004GSI2OS,None,Usually ships in 6-10 business days,now,FeatherLite,Onyx Black/ Stone,NaN,,$26.26,https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,None,SHIRT,None,"[False, https://www.amazon.com/reviews/iframe?...",None,https://images-na.ssl-images-amazon.com/images...,Featherlite Ladies' Long Sleeve Stain Resistan...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183138 entries, 0 to 183137
Data columns (total 19 columns):
asin                 183138 non-null object
author               1 non-null object
availability         24532 non-null object
availability_type    24559 non-null object
brand                182987 non-null object
color                64956 non-null object
editorial_reivew     180380 non-null object
editorial_review     2758 non-null object
formatted_price      28395 non-null object
large_image_url      183138 non-null object
manufacturer         42899 non-null object
medium_image_url     183138 non-null object
model                62370 non-null object
product_type_name    183138 non-null object
publisher            42899 non-null object
reviews              183138 non-null object
sku                  363 non-null object
small_image_url      183138 non-null object
title                183138 non-null object
dtypes: object(19)
memory usage: 26.5+ MB


In [5]:
df = df[['asin', 'brand', 'color', 'medium_image_url','product_type_name', 'title', 'formatted_price']]

In [6]:
df.brand.describe()

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object

In [7]:
df.brand.value_counts().head(10)

Zago                         223
XQS                          222
Yayun                        215
YUNY                         198
XiaoTianXin-women clothes    193
Generic                      192
Boohoo                       190
Alion                        188
TheMogan                     187
Abetteric                    187
Name: brand, dtype: int64

In [8]:
df.color.describe()

count     64956
unique     7380
top       Black
freq      13207
Name: color, dtype: object

In [9]:
df.color.value_counts().head(5)

Black    13207
White     8616
Blue      3570
Red       2289
Pink      1842
Name: color, dtype: int64

In [10]:
len(df['color'].isnull())

183138

In [11]:
df['product_type_name'].describe()

count     183138
unique        72
top        SHIRT
freq      167794
Name: product_type_name, dtype: object

In [12]:
df.product_type_name.value_counts().head(10)

SHIRT                         167794
APPAREL                         3549
BOOKS_1973_AND_LATER            3336
DRESS                           1584
SPORTING_GOODS                  1281
SWEATER                          837
OUTERWEAR                        796
OUTDOOR_RECREATION_PRODUCT       729
ACCESSORY                        636
UNDERWEAR                        425
Name: product_type_name, dtype: int64

In [13]:
df['title'].describe()

count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object

In [14]:
df['formatted_price'].describe()

count      28395
unique      3135
top       $19.99
freq         945
Name: formatted_price, dtype: object

In [15]:
df = df.loc[~df['formatted_price'].isnull()]
print('Number of data points :', df.shape)

Number of data points : (28395, 7)


In [16]:
df =df.loc[~df['color'].isnull()]
print('Number of data :', df.shape)

Number of data : (28385, 7)


In [17]:
df =df.loc[~df['title'].isnull()]
print('Number of data points :', df.shape)

Number of data points : (28385, 7)


In [19]:
sum(df.duplicated('title'))

2325

In [23]:
df[df['asin'].str.contains('B00AQ4GM')==True]

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
21150,B00AQ4GM1G,Red Kap,BB,https://images-na.ssl-images-amazon.com/images...,OUTERWEAR,"Red Kap Lee Women's Basic Pique Polo, Women, 5...",$20.79
22628,B00AQ4GMTS,Red Kap,BR,https://images-na.ssl-images-amazon.com/images...,OUTERWEAR,"Red Kap Lee Women's Basic Pique Polo, Women, 5...",$18.72
35672,B00AQ4GMLQ,Red Kap,BR,https://images-na.ssl-images-amazon.com/images...,OUTERWEAR,"Red Kap Lee Women's Basic Pique Polo, Women, 5...",$17.46
45145,B00AQ4GMCK,Red Kap,BR,https://images-na.ssl-images-amazon.com/images...,OUTERWEAR,"Red Kap Lee Women's Basic Pique Polo, Women, 5...",$17.46


In [26]:
df_long = df[df['title'].apply(lambda x: len(x.split())>4)]
print("After removal of products with short description:", df_long.shape[0])

After removal of products with short description: 27949


In [27]:
df_long.sort_values('title',inplace=True, ascending=False)
df_long.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
61973,B06Y1KZ2WB,Éclair,Black/Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,Éclair Women's Printed Thin Strap Blouse Black...,$24.99
133820,B010RV33VE,xiaoming,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19
81461,B01DDSDLNS,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Women's White Long Sleeve Single Brea...,$21.58
75995,B00X5LYO9Y,xiaoming,Red Anchors,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91
151570,B00WPJG35K,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32


In [28]:
indices = []
for i,row in df_long.iterrows():
    indices.append(i)

In [29]:
import itertools
dupe_asins = []
i = 0
j = 0
num_data_points = df_long.shape[0]
while i < num_data_points and j < num_data_points:   
    previous_i = i
    # store the list of words of ith string in a
    a = df['title'].loc[indices[i]].split()
    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:
        # store the list of words of jth string in b
        b = df['title'].loc[indices[j]].split()
        # store the maximum length of two strings
        length = max(len(a), len(b))
        # count is used to store the number of words that are matched in both strings
        count  = 0

        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words we include the 1st string index
            dupe_asins.append(df_long['asin'].loc[indices[i]])

            # if the comaprision between is between num_data_points, num_data_points-1 strings and they differ in more than 2 words we include both
            if j == num_data_points-1: dupe_asins.append(df_long['asin'].loc[indices[j]])

            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break

In [30]:
df = df.loc[df['asin'].isin(dupe_asins)]

In [31]:
df.shape

(17593, 7)

In [33]:
df.to_pickle('apparel_data_small')

In [36]:
# we use the list of stop words that are downloaded from nltk lib.
stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalnum()))
            # Conver all letters to lower-case
            word = word.lower()
            # stop-word removal
            if not word in stop_words:
                string += word + " "
        df[column][index] = string

list of stop words: {'re', 'he', 'or', 'wasn', 'ain', "wouldn't", 'its', 'while', 'shouldn', 'there', 'myself', 'into', "doesn't", "she's", "hadn't", 'me', 'between', 'don', 'than', 'those', 'yourselves', "you're", 'isn', "needn't", 'above', 'whom', 'once', "shan't", "that'll", 'y', 'further', "aren't", 'again', 'nor', "didn't", 'ma', 'it', 's', 'won', 'if', 'mightn', 'but', 'is', 'herself', 'themselves', 'which', 'such', "shouldn't", 'o', 't', 'up', 'then', 'and', 'now', 'any', 'from', 'under', "should've", 'my', 'an', "won't", 'didn', 'before', 'in', "couldn't", 'here', 'their', "you'll", 'during', 'to', "mustn't", 'will', "you've", 'yourself', "it's", 'by', 'most', 'that', 'who', 'having', 'each', 'some', 'had', 'm', 'for', 'same', 'down', 'his', 'they', 'through', 'does', 'why', "mightn't", 'after', 'of', 'weren', 'own', "haven't", 'just', 'these', 'until', "isn't", 'mustn', 'did', 'am', 'on', 'aren', 'theirs', 'out', 'ourselves', 'about', 'them', 'only', 'we', 'other', "don't", 'a

In [37]:
# we take each title and we text-preprocess it.
for index, row in df.iterrows():
    nlp_preprocessing(row['title'], index, 'title')

In [39]:
df.to_pickle('apperal_data_preprocessed')

In [40]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
print(stemmer.stem('arguing'))


argu
fish


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer = CountVectorizer()
title_features   = title_vectorizer.fit_transform(df['title'])
title_features.get_shape()

(17593, 12927)

In [57]:
def bag_of_words_model(doc_id, num_results):
    pairwise_dist = pairwise_distances(title_features,title_features[doc_id])
    # returns indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
    data frame indices of the 9 smallest distace's
    df_indices = list(df.index[indices])
    
    for i in range(0,len(indices)):
        print('ASIN :',df['asin'].loc[df_indices[i]])
        print ('Brand:', df['brand'].loc[df_indices[i]])
        print ('Title:', df['title'].loc[df_indices[i]])
        print ('Image:', df['medium_image_url'].loc[df_indices[i]])
        print ('Euclidean similarity with the query image :', pdists[i])
        print('='*60)

#call the bag-of-words model for a product to get similar products.
bag_of_words_model(666, 10)


ASIN : B00YONC7AU
Brand: AutumnFall
Title: autumnfall woman sleeveless vneck candy vest loose tank tops tshirt 
Image: https://images-na.ssl-images-amazon.com/images/I/31dys31f4tL._SL160_.jpg
Euclidean similarity with the query image : 0.0
ASIN : B00YBYZ6Q4
Brand: WenHong
Title: wenhong women cute sleeveless vneck vest loose tank tops tshirt 
Image: https://images-na.ssl-images-amazon.com/images/I/51CDPWB5-4L._SL160_.jpg
Euclidean similarity with the query image : 2.449489742783178
ASIN : B014F88CT8
Brand: Black Temptation
Title: sunflower leisure vest comfortable tank tops chiffon sleeveless tshirt 
Image: https://images-na.ssl-images-amazon.com/images/I/51ge2Ze2%2B-L._SL160_.jpg
Euclidean similarity with the query image : 3.0
ASIN : B01IENQA1C
Brand: JUNKE
Title: junke summer woman lady sleeveless vneck candy vest loose tank tops 3 pack 3 pack 
Image: https://images-na.ssl-images-amazon.com/images/I/41UgLHWmzmL._SL160_.jpg
Euclidean similarity with the query image : 3.0
ASIN : B00XTL

In [60]:
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_title_features = tfidf_title_vectorizer.fit_transform(df['title'])

In [62]:
def tfidf_model(doc_id, num_results):
    pairwise_dist = pairwise_distances(tfidf_title_features,tfidf_title_features[doc_id])
    # np.argsort will return indices of smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
    #data frame indices of the 9 smallest distace's
    df_indices = list(df.index[indices])

    for i in range(0,len(indices)):
        print('ASIN :',df['asin'].loc[df_indices[i]])
        print('BRAND :',df['brand'].loc[df_indices[i]])
        print ('Image:', df['medium_image_url'].loc[df_indices[i]])
        print ('Eucliden distance from the given image :', pdists[i])
        print('='*125)
tfidf_model(666, 20)

ASIN : B00YONC7AU
BRAND : AutumnFall
Image: https://images-na.ssl-images-amazon.com/images/I/31dys31f4tL._SL160_.jpg
Eucliden distance from the given image : 0.0
ASIN : B01IENQA1C
BRAND : JUNKE
Image: https://images-na.ssl-images-amazon.com/images/I/41UgLHWmzmL._SL160_.jpg
Eucliden distance from the given image : 0.9940536355903371
ASIN : B07543MPXY
BRAND : Rbwinner
Image: https://images-na.ssl-images-amazon.com/images/I/41ESPu1pwTL._SL160_.jpg
Eucliden distance from the given image : 1.0800725833224063
ASIN : B00YBYZ6Q4
BRAND : WenHong
Image: https://images-na.ssl-images-amazon.com/images/I/51CDPWB5-4L._SL160_.jpg
Eucliden distance from the given image : 1.1003337374613402
ASIN : B07549N5F4
BRAND : Rbwinner
Image: https://images-na.ssl-images-amazon.com/images/I/31VHkZ9EP-L._SL160_.jpg
Eucliden distance from the given image : 1.1169549818005586
ASIN : B00WR3W448
BRAND : AutumnFall
Image: https://images-na.ssl-images-amazon.com/images/I/41mfjfsikOL._SL160_.jpg
Eucliden distance from th

In [63]:
df.to_pickle('apparel_data_recommendation')